In [ ]:
!pip install statsforecast

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns # Import seaborn
from statsforecast import StatsForecast
from statsforecast.models import AutoARIMA
from prophet import Prophet
from sklearn.metrics import mean_absolute_error, mean_squared_error
import os # Import os

# Configurar seaborn
sns.set_style("whitegrid")
plt.rcParams['figure.figsize'] = (12, 6)

# Cargar el archivo CSV
file_path = '/content/drive/MyDrive/Paul Lora - Tesis/Tesis P/full_publication_data.csv'

if os.path.exists(file_path):
    df = pd.read_csv(file_path)
    print(f"¡Éxito! El archivo {file_path} se ha cargado correctamente.")
    # Opcional: mostrar las primeras filas
    # display(df.head())
else:
    print(f"Error: El archivo {file_path} no se encontró. Por favor, verifica la ruta.")

¡Éxito! El archivo /content/drive/MyDrive/Paul Lora - Tesis/Tesis P/full_publication_data.csv se ha cargado correctamente.


## Fase 2: Preprocesamiento de Datos y Modelado

### 2.1 Preprocesamiento de Datos
Here, we will prepare the data for time series forecasting. This involves filtering the data for a specific university and aggregating the number of publications by year.

In [ ]:
# Filter data for a specific university (example: Escuela Politécnica Nacional)
university_name = "Escuela Politécnica Nacional"
df_university = df[df['affiliation_name'] == university_name].copy()

# Convert 'publication_date' to datetime and extract the year
df_university['publication_date'] = pd.to_datetime(df_university['publication_date'], errors='coerce')
df_university['year'] = df_university['publication_date'].dt.year

# Aggregate publications by year
ts_university = df_university.groupby('year').size().reset_index(name='y')

# Rename columns for forecasting libraries
ts_university = ts_university.rename(columns={'year': 'ds'})

# Display the resulting time series data
display(ts_university.head())
display(ts_university.tail())

,ds,y
0,1935,2
1,1936,2
2,1938,1
3,1942,1
4,1954,1


,ds,y
51,2018,1115
52,2019,1165
53,2020,1379
54,2021,1181
55,2022,615


### 2.2 División de Datos (Train/Test Split)
Now, we will split the time series data into training and testing sets to evaluate the performance of our forecasting models. We will use data up to a certain year for training and the remaining data for testing.

In [ ]:
# Define the split year
split_year = 2020

# Split data into training and testing sets
train_data = ts_university[ts_university['ds'] < split_year]
test_data = ts_university[ts_university['ds'] >= split_year]

# Display the shapes of the training and testing sets
print("Training data shape:", train_data.shape)
print("Testing data shape:", test_data.shape)

# Display the last few rows of the training data and the first few rows of the testing data
print("\nTraining data tail:")
display(train_data.tail())
print("\nTesting data head:")
display(test_data.head())

Training data shape: (53, 2)
Testing data shape: (3, 2)

Training data tail:


,ds,y
48,2015,281
49,2016,478
50,2017,691
51,2018,1115
52,2019,1165



Testing data head:


,ds,y
53,2020,1379
54,2021,1181
55,2022,615


### 2.3 Entrenamiento y Evaluación de Modelos
In this section, we will train different time series forecasting models using the training data and evaluate their performance on the testing data.

In [ ]:
import pandas as pd

# Asumiendo que 'df' es tu DataFrame cargado con la estructura que mostraste
# y 'publication_date' es de tipo datetime.
df['publication_date'] = pd.to_datetime(df['publication_date'], errors='coerce') # Convert to datetime
df['year'] = df['publication_date'].dt.year

# Filtrar para tener un histórico consistente (desde el punto de inflexión 2010)
df_modelar = df[df['year'] >= 2010].copy()

# Agrupar por año y por nombre de afiliación para crear las características
# Usaremos 'affiliation_name' como el identificador de la universidad
features_df = df_modelar.groupby(['year', 'affiliation_name']).agg(
    publication_count=('article_id', 'nunique'),
    distinct_authors=('author_id', 'nunique')
).reset_index()

# --- La clave del modelo ---
# Crear el target: El número de publicaciones del SIGUIENTE año para ESA MISMA universidad.
# Usamos groupby().shift() para evitar mezclar datos entre universidades.
features_df['target'] = features_df.groupby('affiliation_name')['publication_count'].shift(-1)

# Eliminar las filas donde el target es NaN (corresponde al último año de nuestro dataset)
features_df.dropna(subset=['target'], inplace=True)

# Convertir el target a entero
features_df['target'] = features_df['target'].astype(int)

print("DataFrame listo para el modelo:")
display(features_df.head())

DataFrame listo para el modelo:


,year,affiliation_name,publication_count,distinct_authors,target
0,2010,Academia Cotopaxi,1,1,1
1,2010,Agencia de Bioseguridad de Galápagos,2,1,4
2,2010,Agencia de Regulacion y Control de La Biosegur...,2,1,4
3,2010,Agencia de Regulación y Control de la Biosegur...,2,1,4
4,2010,Agencia de Regulación y Control de la Biosegur...,2,1,4


### 2.2 División de Datos (Train/Test Split for Granular Model)
Now, we will split the `features_df` into training and testing sets to evaluate the performance of our granular forecasting model. We will use data up to a certain year for training and the remaining data for testing.

In [ ]:
# Define the split year for the granular model
granular_split_year = 2022 # Using 2022 to predict 2023, as target is next year's publications

# Split data into training and testing sets for the granular model
train_granular = features_df[features_df['year'] < granular_split_year].copy()
test_granular = features_df[features_df['year'] >= granular_split_year].copy()

# Display the shapes of the training and testing sets
print("Granular Training data shape:", train_granular.shape)
print("Granular Testing data shape:", test_granular.shape)

# Display the last few rows of the training data and the first few rows of the testing data
print("\nGranular Training data tail:")
display(train_granular.tail())
print("\nGranular Testing data head:")
display(test_granular.head())

Granular Training data shape: (12586, 5)
Granular Testing data shape: (4, 5)

Granular Training data tail:


,year,affiliation_name,publication_count,distinct_authors,target
15646,2021,Yachay EP,4,1,1
15648,2021,Yachay University for Experimental Technology ...,290,261,134
15655,2021,Zurita &amp; Zurita Laboratorios,10,6,6
15662,2021,Área de Investigación y Monitoreo de Avifauna,5,6,1
15665,2021,“Laica Eloy Alfaro de Manabí” University,1,3,1



Granular Testing data head:


,year,affiliation_name,publication_count,distinct_authors,target
16047,2022,Escuela Superior Politecnica del Litoral Ecuador,321,374,1
16888,2022,Technical University of Ambato,43,47,1
17031,2022,Universidad Técnica de Ambato,132,179,2
17091,2022,University of Cuenca,170,250,1


### 2.3 Entrenamiento y Evaluación de Modelos (Enfoque Granular)
Now, we will train and evaluate forecasting models using the granular training and testing data.

In [ ]:
# Celda de Código para Entrenamiento
import lightgbm as lgb
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
import joblib # Para guardar el modelo y el encoder

# Preparar los datos para LightGBM
# 1. Convertir 'affiliation_name' a una categoría numérica
le = LabelEncoder()
features_df['affiliation_encoded'] = le.fit_transform(features_df['affiliation_name'])

# Definir características (X) y objetivo (y)
X = features_df[['year', 'affiliation_encoded', 'publication_count', 'distinct_authors']]
y = features_df['target']

# Dividir en entrenamiento y prueba
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Entrenar el modelo final con todos los datos
final_model = lgb.LGBMRegressor(random_state=42)
final_model.fit(X, y)

print("Modelo entrenado exitosamente.")

# --- Guardar artefactos para producción ---
# Guardamos el modelo entrenado y el LabelEncoder para usarlo en el backend
joblib.dump(final_model, 'publication_model.pkl')
joblib.dump(le, 'affiliation_encoder.pkl')

print("Modelo y encoder guardados en 'publication_model.pkl' y 'affiliation_encoder.pkl'")

[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.001376 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 517
[LightGBM] [Info] Number of data points in the train set: 12590, number of used features: 4
[LightGBM] [Info] Start training from score 9.268388
Modelo entrenado exitosamente.
Modelo y encoder guardados en 'publication_model.pkl' y 'affiliation_encoder.pkl'


### 2.4 Evaluación del Modelo (Enfoque Granular)
Now we will evaluate the performance of the trained LightGBM model on the test data.

In [ ]:
from sklearn.metrics import mean_absolute_error, mean_squared_error
import numpy as np # Import numpy

# Make predictions on the test data
predictions = final_model.predict(X_test)

# Evaluate the model
mae = mean_absolute_error(y_test, predictions)
mse = mean_squared_error(y_test, predictions)
rmse = np.sqrt(mse) # Calculate RMSE by taking the square root of MSE

print(f"Mean Absolute Error (MAE): {mae:.2f}")
print(f"Mean Squared Error (MSE): {mse:.2f}")
print(f"Root Mean Squared Error (RMSE): {rmse:.2f}")

Mean Absolute Error (MAE): 2.66
Mean Squared Error (MSE): 35.23
Root Mean Squared Error (RMSE): 5.94


### 3.1 Carga del Modelo y Predicción (Ejemplo para Backend)
Aquí mostramos un ejemplo de cómo cargar el modelo LightGBM y el LabelEncoder guardados, y utilizarlos para realizar una predicción basada en datos de entrada simulados, similar a cómo lo harías en un backend.

In [ ]:
import joblib
import pandas as pd
import numpy as np # Importar numpy

# Ruta a los archivos guardados
model_path = 'publication_model.pkl'
encoder_path = 'affiliation_encoder.pkl'

# Cargar el modelo y el encoder
try:
    loaded_model = joblib.load(model_path)
    loaded_encoder = joblib.load(encoder_path)
    print("Modelo y encoder cargados exitosamente.")
except FileNotFoundError:
    print(f"Error: Asegúrate de que los archivos '{model_path}' y '{encoder_path}' existan en la ruta especificada.")
    # Puedes agregar aquí una lógica para manejar el error, como salir o pedir al usuario que verifique las rutas.
    loaded_model = None
    loaded_encoder = None


if loaded_model and loaded_encoder:
    # --- Simulación de datos de entrada recibidos en el backend ---
    # Estos datos vendrían de tu frontend o de otra fuente
    input_data = {
      "year": 2024,
      "affiliation_name": "Escuela Politécnica Nacional", # El nombre de la universidad
      "publication_count": 615, # Publicaciones del año anterior (2023 en este ejemplo)
      "distinct_authors": 374 # Autores distintos del año anterior (2023 en este ejemplo)
    }

    # --- Preprocesamiento de los datos de entrada (como se haría en el backend) ---

    # 1. Codificar el nombre de la afiliación
    try:
        affiliation_encoded = loaded_encoder.transform([input_data['affiliation_name']])[0]
    except ValueError:
        print(f"Error: El nombre de afiliación '{input_data['affiliation_name']}' no fue visto durante el entrenamiento. No se puede codificar.")
        affiliation_encoded = None # O manejar este caso de otra manera

    if affiliation_encoded is not None:
        # 2. Crear un DataFrame con las características en el formato correcto
        # Asegúrate de que los nombres de las columnas coincidan con los usados durante el entrenamiento
        features_for_prediction = pd.DataFrame({
            'year': [input_data['year']],
            'affiliation_encoded': [affiliation_encoded],
            'publication_count': [input_data['publication_count']],
            'distinct_authors': [input_data['distinct_authors']]
        })

        # 3. Realizar la predicción
        predicted_publications = loaded_model.predict(features_for_prediction)

        # El resultado de predict es un array, tomamos el primer elemento
        predicted_count = max(0, int(np.round(predicted_publications[0]))) # Aseguramos que la predicción sea un entero no negativo

        print(f"\nDatos de entrada para la predicción:\n{features_for_prediction}")
        print(f"\nPredicción para {input_data['affiliation_name']} en {input_data['year']}: {predicted_count} publicaciones.")

    else:
        print("\nNo se pudo realizar la predicción debido a un error de codificación de la afiliación.")

Modelo y encoder cargados exitosamente.

Datos de entrada para la predicción:
   year  affiliation_encoded  publication_count  distinct_authors
0  2024                  694                615               374

Predicción para Escuela Politécnica Nacional en 2024: 422 publicaciones.


### 3.2 Prueba del Modelo con Datos Históricos
Vamos a usar datos de un año anterior para probar el modelo entrenado y comparar la predicción con el valor real.

In [ ]:
# Seleccionar un ejemplo de datos históricos para la prueba
# Tomaremos una fila del conjunto de entrenamiento granular
# Asegúrate de que train_granular exista (se creó en un paso anterior)

if 'features_df' in locals() and not features_df.empty:
    # Filter features_df to include only data that would have been in the training set
    trainable_features_df = features_df[features_df['year'] < granular_split_year].copy()

    if not trainable_features_df.empty:
        # Seleccionamos una fila de ejemplo aleatoria de los datos que serían de entrenamiento
        historical_data_row_df = trainable_features_df.sample(1)

        # Extraer los datos de entrada para la predicción
        input_year = historical_data_row_df['year'].iloc[0]
        input_affiliation_name = historical_data_row_df['affiliation_name'].iloc[0] # Get the original name
        input_affiliation_encoded = historical_data_row_df['affiliation_encoded'].iloc[0]
        input_publication_count = historical_data_row_df['publication_count'].iloc[0]
        input_distinct_authors = historical_data_row_df['distinct_authors'].iloc[0]
        real_target_value = historical_data_row_df['target'].iloc[0] # The real value we want to predict


        # Crear el DataFrame para la predicción (año para el que queremos predecir)
        # El modelo predice el AÑO SIGUIENTE, por lo que el 'year' de entrada debe ser el año de la fila histórica.
        features_for_historical_prediction = pd.DataFrame({
            'year': [input_year],
            'affiliation_encoded': [input_affiliation_encoded],
            'publication_count': [input_publication_count],
            'distinct_authors': [input_distinct_authors]
        })

        # Realizar la predicción
        predicted_publications_historical = loaded_model.predict(features_for_historical_prediction)
        predicted_count_historical = max(0, int(np.round(predicted_publications_historical[0])))

        # Obtener el nombre de la afiliación para mostrarlo
        affiliation_name_historical = loaded_encoder.inverse_transform([input_affiliation_encoded])[0]


        print(f"Datos históricos utilizados para la predicción (Año: {input_year}, Universidad: {affiliation_name_historical}):")
        display(features_for_historical_prediction)
        print(f"\nValor REAL de publicaciones en el año siguiente ({input_year + 1}): {real_target_value}")
        print(f"Valor PREDECIDO de publicaciones para el año siguiente ({input_year + 1}): {predicted_count_historical}")

        # Calcular el error absoluto para este ejemplo específico
        error_absolute = abs(real_target_value - predicted_count_historical)
        print(f"Error absoluto para esta predicción: {error_absolute}")

    else:
        print("Error: No hay datos históricos disponibles en 'features_df' antes del año de división.")

else:
    print("Error: La variable 'features_df' no se encontró o está vacía. Asegúrate de haber ejecutado los pasos de preprocesamiento.")

Datos históricos utilizados para la predicción (Año: 2020, Universidad: Yanayacu Biological Station &amp; Center for Creative Studies):


,year,affiliation_encoded,publication_count,distinct_authors
0,2020,2689,5,1



Valor REAL de publicaciones en el año siguiente (2021): 1
Valor PREDECIDO de publicaciones para el año siguiente (2021): 3
Error absoluto para esta predicción: 2


### 3.3 Prueba del Modelo con Datos Históricos (Iteración por Año)
Vamos a iterar a través de los datos históricos desde 2010 hasta 2020 para ver las predicciones del modelo y compararlas con los valores reales para cada universidad en cada año.

In [ ]:
# Asegúrate de que features_df, loaded_model y loaded_encoder existan
if 'features_df' in locals() and not features_df.empty and loaded_model and loaded_encoder:
    # Definir el rango de años para la prueba
    start_year = 2010
    end_year = 2020

    # Filtrar los datos para el rango de años especificado
    historical_data_range_df = features_df[(features_df['year'] >= start_year) & (features_df['year'] <= end_year)].copy()

    if not historical_data_range_df.empty:
        print(f"Realizando predicciones y recopilando resultados para los años {start_year} a {end_year}...\n")

        # Lista para almacenar los resultados
        results_list = []

        # Iterar sobre cada fila en el DataFrame filtrado
        for index, row in historical_data_range_df.iterrows():
            input_year = row['year']
            input_affiliation_name = row['affiliation_name']
            input_affiliation_encoded = row['affiliation_encoded']
            input_publication_count = row['publication_count']
            input_distinct_authors = row['distinct_authors']
            real_target_value = row['target'] # El valor real que queremos predecir (publicaciones del año siguiente)

            # Crear el DataFrame para la predicción (año para el que queremos predecir)
            # El modelo predice el AÑO SIGUIENTE, por lo que el 'year' de entrada debe ser el año de la fila histórica.
            features_for_prediction = pd.DataFrame({
                'year': [input_year],
                'affiliation_encoded': [input_affiliation_encoded],
                'publication_count': [input_publication_count],
                'distinct_authors': [input_distinct_authors]
            })

            # Realizar la predicción
            predicted_publications = loaded_model.predict(features_for_prediction)
            predicted_count = max(0, int(np.round(predicted_publications[0]))) # Aseguramos que la predicción sea un entero no negativo

            # Calcular el error absoluto para este ejemplo específico
            error_absolute = abs(real_target_value - predicted_count)

            # Agregar los resultados a la lista
            results_list.append({
                'Año Predicción': input_year + 1,
                'Año Datos Entrada': input_year,
                'Universidad': input_affiliation_name,
                'Valor Real': real_target_value,
                'Valor Predecido': predicted_count,
                'Error Absoluto': error_absolute
            })

        # Crear el DataFrame a partir de la lista de resultados
        results_df = pd.DataFrame(results_list)

        print("Resultados de las predicciones históricas:")
        display(results_df.head())
        display(results_df.tail())
        print(f"\nDataFrame de resultados creado con {len(results_df)} filas.")


    else:
        print(f"No se encontraron datos en 'features_df' para el rango de años {start_year} a {end_year}.")

else:
    print("Error: Asegúrate de que las variables 'features_df', 'loaded_model' y 'loaded_encoder' existan y no estén vacías.")

Realizando predicciones y recopilando resultados para los años 2010 a 2020...

Resultados de las predicciones históricas:


,Año Predicción,Año Datos Entrada,Universidad,Valor Real,Valor Predecido,Error Absoluto
0,2011,2010,Academia Cotopaxi,1,2,1
1,2011,2010,Agencia de Bioseguridad de Galápagos,4,2,2
2,2011,2010,Agencia de Regulacion y Control de La Biosegur...,4,2,2
3,2011,2010,Agencia de Regulación y Control de la Biosegur...,4,2,2
4,2011,2010,Agencia de Regulación y Control de la Biosegur...,4,2,2


,Año Predicción,Año Datos Entrada,Universidad,Valor Real,Valor Predecido,Error Absoluto
11470,2021,2020,Área de Investigación y Monitoreo de Avifauna,5,2,3
11471,2021,2020,“Eloy Alfaro” Lay University of Manabí,2,3,1
11472,2021,2020,“Empresa Eléctrica Regional Centro Sur C.A.”—C...,2,2,0
11473,2021,2020,“Epidemiología perinatal,1,2,1
11474,2021,2020,“Laica Eloy Alfaro de Manabí” University,1,2,1



DataFrame de resultados creado con 11475 filas.


In [ ]:
# Guardar el DataFrame de resultados en un archivo CSV
results_df.to_csv('historical_predictions_results.csv', index=False)

print("DataFrame 'results_df' guardado en 'historical_predictions_results.csv'")

DataFrame 'results_df' guardado en 'historical_predictions_results.csv'
